In [8]:
# !pip install tensorflow

In [9]:
# !pip install tensorflow>=2.15.0

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow


In [ ]:
# pip install tqdm>=4.66

### Loading Pre-Trained weights from OpenAI

- Download the weights from [kaggle](https://www.kaggle.com/datasets/xhlulu/openai-gpt2-weights)
- Or use gpt_download3.py code file: [gpt_download3.py](https://drive.google.com/file/d/1JX_DNJ9Xvty_ItciHhl_3TyOfahkGvvI/view)

In [53]:
import tensorflow as tf
import tqdm
import numpy as np

print("NumPy version: ", np.__version__)
print('TensorFlow version:', tf.__version__)
print('tqdm version:', tqdm.__version__)

NumPy version:  2.0.2
TensorFlow version: 2.18.0
tqdm version: 4.67.0


In [29]:
import torch
import torch.nn as nn

In [16]:
# hepls us download the 7 files from gpt2 124M
from gpt_download3 import download_and_load_gpt2


settings, params = download_and_load_gpt2(model_size='124M', models_dir='gpt2')

c:\Users\pc\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
models\124M\checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 15.4kiB/s]
c:\Users\pc\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
models\124M\encoder.json: 100%|██████████| 1.04M/1.04M [00:01<00:00, 718kiB/s]
c:\Users\pc\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning:

In [38]:
print("Settings: ", settings)
print("\nParameter dictionary key: ", params.keys())

Settings:  {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}

Parameter dictionary key:  dict_keys(['blocks', 'b', 'g', 'wpe', 'wte'])


In [36]:
print(params['wte'])
print("Token embeddings weight tensor dim", params['wte'].shape)

[[-0.11010301 -0.03926672  0.03310751 ... -0.1363697   0.01506208
   0.04531523]
 [ 0.04034033 -0.04861503  0.04624869 ...  0.08605453  0.00253983
   0.04318958]
 [-0.12746179  0.04793796  0.18410145 ...  0.08991534 -0.12972379
  -0.08785918]
 ...
 [-0.04453601 -0.05483596  0.01225674 ...  0.10435229  0.09783269
  -0.06952604]
 [ 0.1860082   0.01665728  0.04611587 ... -0.09625227  0.07847701
  -0.02245961]
 [ 0.05135201 -0.02768905  0.0499369  ...  0.00704835  0.15519823
   0.12067825]]
Token embeddings weight tensor dim (50257, 768)


In [37]:
# GPT-2 small config
GPT_CONFIG_124M = {
    "vocab_size" : 50257,
    "context_length" : 256, #1024,
    "emb_dim" : 768,
    "n_heads" : 12,
    "n_layers" : 12,
    "drop_rate" : 0.1,
    "qkv_bias" : False
}

In [33]:
# Define model configurations in a dictionary compactness

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25}
}

# Copy the base connfiguration and update with specific model settings
model_name = "gpt2-small (124M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])

In [34]:
from modules import GPTModel

NEW_CONFIG.update({'context_length': 1024, 'qkv_bias': True})
gpt = GPTModel(NEW_CONFIG)
gpt.eval();

In [35]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shapes mismatch. Left: {left.shape}, Right: {right.shape}")
    return torch.nn.Parameter(torch.tensor(right))

In [43]:
def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])

    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(
            (params['blocks'][b]['attn']['c_attn'])['w'], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign(
            gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign(
            gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign(
            gpt.trf_blocks[b].att.W_value.weight, v_w.T)
        
        q_b, k_b, v_b = np.split(
            (params['blocks'][b]['attn']['c_attn'])['b'], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign(
            gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign(
            gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign(
            gpt.trf_blocks[b].att.W_value.bias, v_b)
        
        gpt.trf_blocks[b].att.out_proj.weight = assign(
            gpt.trf_blocks[b].att.out_proj.weight, 
            params['blocks'][b]['attn']['c_proj']['w'].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign(
            gpt.trf_blocks[b].att.out_proj.bias, 
            params['blocks'][b]['attn']['c_proj']['b'])
        
        gpt.trf_blocks[b].ff.layers[0].weight = assign(
            gpt.trf_blocks[b].ff.layers[0].weight, 
            params['blocks'][b]['mlp']['c_fc']['w'].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign(
            gpt.trf_blocks[b].ff.layers[0].bias, 
            params['blocks'][b]['mlp']['c_fc']['b'])
        gpt.trf_blocks[b].ff.layers[2].weight = assign(
            gpt.trf_blocks[b].ff.layers[2].weight, 
            params['blocks'][b]['mlp']['c_proj']['w'].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign(
            gpt.trf_blocks[b].ff.layers[2].bias, 
            params['blocks'][b]['mlp']['c_proj']['b'])

        gpt.trf_blocks[b].norm1.scale = assign(
            gpt.trf_blocks[b].norm1.scale, 
            params['blocks'][b]['ln_1']['g'])
        gpt.trf_blocks[b].norm1.shift = assign(
            gpt.trf_blocks[b].norm1.shift, 
            params['blocks'][b]['ln_1']['b'])
        gpt.trf_blocks[b].norm2.scale = assign(
            gpt.trf_blocks[b].norm2.scale, 
            params['blocks'][b]['ln_2']['g'])
        gpt.trf_blocks[b].norm2.shift = assign(
            gpt.trf_blocks[b].norm2.shift, 
            params['blocks'][b]['ln_2']['b'])
        
    gpt.final_norm.scale = assign(gpt.final_norm.scale, params['g'])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params['b'])
    gpt.out_head.weight = assign(gpt.out_head.weight, params['wte'])
        

In [44]:
load_weights_into_gpt(gpt, params)

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
gpt.to(device);

In [47]:
import torch

# the same func in 8-Lec27
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):
    # get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        # filter logits with tok_k sampling
        if top_k is not None:
            # keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float('-inf')).to(logits.device), logits)

        # apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # apply softmax to get probas
            probs = torch.softmax(logits, dim=-1) # (batch size, context len)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (batch_size, 1)
        
        # otherwise get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)

        if idx_next == eos_id:
            break # stop generating early if end-of-seq token is encountered

        idx = torch.cat((idx, idx_next), dim=1) # (batch_size, num_tokens+1)
    
    return idx


In [52]:
# the model loaded correctly, lets test it
from modules import text_to_token_ids, token_ids_to_text
import tiktoken


tokenizer = tiktoken.get_encoding("gpt2")
torch.manual_seed(123)

token_ids = generate(
    model=gpt,
    idx=text_to_token_ids("Every effort moves you", tokenizer).to(device),
    max_new_tokens=25,
    context_size=NEW_CONFIG['context_length'],
    top_k=50,
    temperature=1.4
)

print('Output text:\n', token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you toward an equal share for each vote plus half. Inequality is often not an accurate representation of human worth; to know the
